In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from laser_core.propertyset import PropertySet
from laser_core.migration import gravity
from laser_core.migration import row_normalizer
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

from laser_generic import Model
from laser_generic import Infection
from laser_generic import Susceptibility
from laser_generic import Exposure
from laser_generic import Transmission
from laser_generic import Births_ConstantPop
from laser_generic.importation import Infect_Random_Agents
from laser_generic.utils import set_initial_susceptibility_in_patch
from laser_generic.utils import seed_infections_in_patch
import pickle


data_dir = os.path.join('..', "data")
sys.path.append(data_dir)

from EnglandWalesmeasles import data as engwal
distances = np.load(os.path.join("..", "data", "EnglandWalesdistances.npy"))

f"{np.__version__=}"

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


"np.__version__='1.26.4'"

In [2]:
average_pop = np.array([np.mean(place.population) for place in engwal.places.values()])
average_birthrate = np.array([np.mean(place.births/place.population) for place in engwal.places.values()])
average_cbr = 1000*np.sum(average_pop*average_birthrate)/np.sum(average_pop)

average_cbr

17.190759545084912

In [3]:
#For starters, we will work with a constant population and constant crude birth rate over the network.  
#This may prevent seeing some of the periodicity switch behavior, but we are focused on explicitly spatial stuff for now.
data = [{"name": placename,
         "population": place.population[0],
         "latitude": place.latitude,
         "longitude": place.longitude} for placename, place in engwal.places.items()]

scenario = pd.DataFrame(data)
parameters = PropertySet(
    {"seed": 4, "nticks": 365,
    "exp_shape": 40, "exp_scale": 0.25,
    "verbose": True, "beta": 1.2, "inf_mean": 10, "inf_sigma": 2, "cbr": average_cbr,
    "seasonality_factor": 0.2, "seasonality_phase": 182,
    "importation_period": 30, "importation_count": 3, "importation_start": 1, "importation_end": 8*365}
)

model = Model(scenario, parameters)
model.components = [
    Births_ConstantPop,
    Susceptibility,
    Transmission,
    Exposure,
    Infection,
    Infect_Random_Agents,
]

model.patches.network = gravity(np.array(scenario.population), distances, 1e-6, 0, 1, 2)

mu = (1 + parameters.cbr / 1000) ** (1 / 365) - 1
R0 = parameters.beta / (mu + 1 / parameters.inf_mean)
for i in range(len(model.patches)):
    set_initial_susceptibility_in_patch(model, i, 1 / R0 + 0.02 * np.random.normal())

seed_infections_in_patch(model, ipatch=0, ninfections=3)
model.run()

2025-05-14 11:02:15.560948: Creating the generic model…
Initializing the generic model with 954 patches…
2025-05-14 11:02:21.755511: Running the generic model for 365 ticks…


100%|██████████| 365/365 [00:10<00:00, 34.72it/s]

Completed the generic model at 2025-05-14 11:02:32.294423…
Susceptibility_census:       138,946 µs
Transmission_census  :        17,308 µs
Exposure_census      :        13,120 µs
Infection_census     :       294,432 µs
Model                :         3,822 µs
Births_ConstantPop   :       456,994 µs
Susceptibility       :           724 µs
Transmission         :     9,972,136 µs
Exposure             :     4,186,080 µs
Infection            :     3,760,814 µs
Infect_Random_Agents :     2,074,712 µs
Total:                    20,919,088 microseconds


In [ ]:
# Test equality for cases
#cases_equal = np.array_equal(model.patches.cases, model.patches.cases_test[:-1, :])

# Test equality for susceptibility
#susceptibility_equal = np.array_equal(model.patches.susceptibility, model.patches.susceptibility_test[:-1, :])

# Test equality for recovered
#recovered_equal = np.array_equal(model.patches.recovered, model.patches.recovered_test[:-1, :])

# Test equality for exposed
#exposed_equal = np.array_equal(model.patches.exposed, model.patches.exposed_test[:-1, :])

#cases_equal, susceptibility_equal, recovered_equal, exposed_equal

(True, True, True, True)

In [ ]:
data = [{"name": placename,
         "population": place.population[0],
         "latitude": place.latitude,
         "longitude": place.longitude} for placename, place in engwal.places.items()]
scenario = pd.DataFrame(data)
parameters = PropertySet(
    {"seed": 4, "nticks": 50*365,
    "exp_shape": 40, "exp_scale": 0.25,
    "verbose": True, "beta": 1.2, "inf_mean": 10, "inf_sigma": 2, "cbr": average_cbr,
    "seasonality_factor": 0.2, "seasonality_phase": 182,
    "importation_period": 30, "importation_count": 3, "importation_start": 1, "importation_end": 8*365
    }
)

for k in np.logspace(-8, -4, 30):
    model = Model(scenario, parameters)
    model.components = [
        Births_ConstantPop,
        Susceptibility,
        Transmission,
        Exposure,
        Infection,
        Infect_Random_Agents,
    ]

    model.patches.network = row_normalizer(gravity(np.array(scenario.population), distances, k, 0, 1, 2), 0.5)

    mu = (1 + parameters.cbr / 1000) ** (1 / 365) - 1
    R0 = parameters.beta / (mu + 1 / parameters.inf_mean)
    for i in range(len(model.patches)):
        set_initial_susceptibility_in_patch(model, i, 1 / R0 + 0.02 * np.random.normal())

    seed_infections_in_patch(model, ipatch=0, ninfections=3)
    model.run()
    output_path = os.path.join('..', 'outputs', f"EWoutputs_gravity_{k:.1e}.pkl")
    with open(output_path, "wb") as file:
        pickle.dump(model.patches, file)


2025-05-13 23:01:02.898184: Creating the generic model…
Initializing the generic model with 954 patches…
2025-05-13 23:01:11.010602: Running the generic model for 18250 ticks…


100%|██████████| 18250/18250 [11:21<00:00, 26.79it/s]


Completed the generic model at 2025-05-13 23:12:32.211635…
Susceptibility_census:       668,638 µs
Transmission_census  :       180,992 µs
Exposure_census      :       243,684 µs
Infection_census     :       484,212 µs
Model                :       353,234 µs
Births_ConstantPop   :    18,216,846 µs
Susceptibility       :        33,890 µs
Transmission         :   902,747,018 µs
Exposure             :   235,743,964 µs
Infection            :   197,136,478 µs
Infect_Random_Agents :       150,484 µs
Total:                 1,355,959,440 microseconds
2025-05-13 23:12:32.826558: Creating the generic model…
Initializing the generic model with 954 patches…
2025-05-13 23:12:41.292509: Running the generic model for 18250 ticks…


100%|██████████| 18250/18250 [11:31<00:00, 26.40it/s]


Completed the generic model at 2025-05-13 23:24:12.467884…
Susceptibility_census:       549,678 µs
Transmission_census  :       152,862 µs
Exposure_census      :       212,056 µs
Infection_census     :       449,930 µs
Model                :       185,206 µs
Births_ConstantPop   :    18,958,618 µs
Susceptibility       :        33,816 µs
Transmission         :   918,984,244 µs
Exposure             :   238,869,126 µs
Infection            :   197,107,368 µs
Infect_Random_Agents :       151,420 µs
Total:                 1,375,654,324 microseconds
2025-05-13 23:24:12.900430: Creating the generic model…
Initializing the generic model with 954 patches…
2025-05-13 23:24:20.794111: Running the generic model for 18250 ticks…


...

Completed the generic model at 2025-05-14 07:07:14.904647…
Susceptibility_census:       445,042 µs
Transmission_census  :       178,980 µs
Exposure_census      :       174,446 µs
Infection_census     :       427,238 µs
Model                :       297,002 µs
Births_ConstantPop   :    39,190,920 µs
Susceptibility       :        31,354 µs
Transmission         : 1,124,072,778 µs
Exposure             :   409,786,334 µs
Infection            :   263,850,468 µs
Infect_Random_Agents :       138,946 µs
Total:                 1,838,593,508 microseconds
2025-05-14 07:07:15.315382: Creating the generic model…
Initializing the generic model with 954 patches…
2025-05-14 07:07:23.138437: Running the generic model for 18250 ticks…


100%|██████████| 18250/18250 [18:22<00:00, 16.55it/s]


Completed the generic model at 2025-05-14 07:25:45.532969…
Susceptibility_census:       456,526 µs
Transmission_census  :       158,902 µs
Exposure_census      :       182,010 µs
Infection_census     :       449,900 µs
Model                :       140,722 µs
Births_ConstantPop   :    60,163,884 µs
Susceptibility       :        31,514 µs
Transmission         : 1,176,881,614 µs
Exposure             :   606,161,154 µs
Infection            :   352,653,486 µs
Infect_Random_Agents :       137,802 µs
Total:                 2,197,417,514 microseconds
2025-05-14 07:25:45.938522: Creating the generic model…
Initializing the generic model with 954 patches…
2025-05-14 07:25:52.597680: Running the generic model for 18250 ticks…


100%|██████████| 18250/18250 [16:44<00:00, 18.16it/s]


Completed the generic model at 2025-05-14 07:42:37.294772…
Susceptibility_census:       661,120 µs
Transmission_census  :       148,846 µs
Exposure_census      :       229,900 µs
Infection_census     :       421,126 µs
Model                :       328,852 µs
Births_ConstantPop   :    49,605,114 µs
Susceptibility       :        32,290 µs
Transmission         : 1,145,173,110 µs
Exposure             :   504,039,218 µs
Infection            :   301,821,928 µs
Infect_Random_Agents :       140,602 µs
Total:                 2,002,602,106 microseconds


In [6]:
np.logspace(-9, -5, 30)

array([1.00000000e-09, 1.37382380e-09, 1.88739182e-09, 2.59294380e-09,
       3.56224789e-09, 4.89390092e-09, 6.72335754e-09, 9.23670857e-09,
       1.26896100e-08, 1.74332882e-08, 2.39502662e-08, 3.29034456e-08,
       4.52035366e-08, 6.21016942e-08, 8.53167852e-08, 1.17210230e-07,
       1.61026203e-07, 2.21221629e-07, 3.03919538e-07, 4.17531894e-07,
       5.73615251e-07, 7.88046282e-07, 1.08263673e-06, 1.48735211e-06,
       2.04335972e-06, 2.80721620e-06, 3.85662042e-06, 5.29831691e-06,
       7.27895384e-06, 1.00000000e-05])

In [9]:
plt.plot(np.log10(model.patches.populations[0,:]), np.sum(model.patches.incidence[3650:,:]==0, axis=0), 'o')

In [4]:
plt.plot(model.patches.cases_test)

In [17]:
(np.where(model.population.susceptibility == 0))[0]

array([       0,        1,        2, ..., 30182535, 30182536, 30182537])

TypeError: only integer scalar arrays can be converted to a scalar index